# 识别螺丝与垫片
**——基于YOLOv3**

## 环境配置

In [2]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx     # (基于PaddleX静态图版本（低于2.0.0))

## 数据预处理&数据增强

In [3]:
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomDistort(),           # 以一定的概率对图像进行随机像素内容变换
    transforms.RandomExpand(),            # 随机扩张图像
    transforms.RandomCrop(),              # 随机裁剪图像
    transforms.Resize(target_size=608, interp='LINEAR'),      # 线性插值  
    # target_size=608 小了 训练时随机大小影响精度？
    transforms.RandomHorizontalFlip(),    # 随机翻转
    transforms.Normalize(),               # 图像进行标准化
])

eval_transforms = transforms.Compose([
    transforms.Resize(target_size=416, interp='CUBIC'),
    transforms.Normalize(),
])

## 定义数据集Dataset


In [4]:
train_dataset = pdx.datasets.VOCDetection(
    data_dir='voc',
    file_list='voc/train.txt',
    label_list='voc/labels.txt',
    transforms=train_transforms,
    shuffle=False)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='voc',
    file_list='voc/val.txt',
    label_list='voc/labels.txt',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


2022-05-03 10:44:45 [INFO]	Starting to read file list from dataset...
2022-05-03 10:44:46 [INFO]	284 samples in file voc/train.txt
creating index...
index created!
2022-05-03 10:44:46 [INFO]	Starting to read file list from dataset...
2022-05-03 10:44:46 [INFO]	121 samples in file voc/val.txt
creating index...
index created!


## 模型训练


In [5]:
num_classes = len(train_dataset.labels)
model = pdx.det.YOLOv3(num_classes=num_classes, backbone='DarkNet53')
# train_random_shapes (list|tuple): 训练时从列表中随机选择图像大小。默认值为[320, 352, 384, 416, 448, 480, 512, 544, 576, 608]。
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_interval_epochs=30,             # 模型保存间隔
    save_dir='output_yolov3_darknet53',  # 模型保存路径
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:744: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


2022-05-03 10:44:48 [INFO]	Decompressing output_yolov3_darknet53/pretrain/DarkNet53_ImageNet1k_pretrained.tar...


W0503 10:44:49.247629   157 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0503 10:44:49.251977   157 device_context.cc:465] device: 0, cuDNN Version: 7.6.


2022-05-03 10:44:52 [INFO]	Load pretrain weights from output_yolov3_darknet53/pretrain/DarkNet53_ImageNet1k_pretrained.
2022-05-03 10:44:52 [INFO]	There are 260 varaibles in output_yolov3_darknet53/pretrain/DarkNet53_ImageNet1k_pretrained are loaded.


libpng warning: iCCP: known incorrect sRGB profile


...


2022-05-03 13:13:40 [INFO]	[TRAIN] Epoch=270/270, Step=3/35, loss=6.79509, lr=1e-06, time_each_step=0.99s, eta=0:0:45
2022-05-03 13:13:42 [INFO]	[TRAIN] Epoch=270/270, Step=5/35, loss=6.323606, lr=1e-06, time_each_step=0.99s, eta=0:0:43
2022-05-03 13:13:43 [INFO]	[TRAIN] Epoch=270/270, Step=7/35, loss=8.977758, lr=1e-06, time_each_step=1.0s, eta=0:0:41
2022-05-03 13:13:45 [INFO]	[TRAIN] Epoch=270/270, Step=9/35, loss=6.575212, lr=1e-06, time_each_step=1.03s, eta=0:0:40
2022-05-03 13:13:47 [INFO]	[TRAIN] Epoch=270/270, Step=11/35, loss=4.815873, lr=1e-06, time_each_step=1.07s, eta=0:0:39
2022-05-03 13:13:48 [INFO]	[TRAIN] Epoch=270/270, Step=13/35, loss=5.327392, lr=1e-06, time_each_step=1.09s, eta=0:0:37
2022-05-03 13:13:50 [INFO]	[TRAIN] Epoch=270/270, Step=15/35, loss=9.684758, lr=1e-06, time_each_step=1.08s, eta=0:0:35
2022-05-03 13:13:51 [INFO]	[TRAIN] Epoch=270/270, Step=17/35, loss=4.658996, lr=1e-06, time_each_step=1.07s, eta=0:0:33
2022-05-03 13:13:53 [INFO]	[TRAIN] Epoch=270/

libpng warning: iCCP: known incorrect sRGB profile


2022-05-03 13:13:59 [INFO]	[TRAIN] Epoch=270/270, Step=27/35, loss=4.266812, lr=1e-06, time_each_step=0.76s, eta=0:0:20
2022-05-03 13:14:00 [INFO]	[TRAIN] Epoch=270/270, Step=29/35, loss=8.2498, lr=1e-06, time_each_step=0.73s, eta=0:0:18
2022-05-03 13:14:01 [INFO]	[TRAIN] Epoch=270/270, Step=31/35, loss=6.604134, lr=1e-06, time_each_step=0.7s, eta=0:0:16
2022-05-03 13:14:02 [INFO]	[TRAIN] Epoch=270/270, Step=33/35, loss=5.109483, lr=1e-06, time_each_step=0.68s, eta=0:0:15
2022-05-03 13:14:03 [INFO]	[TRAIN] Epoch=270/270, Step=35/35, loss=5.864316, lr=1e-06, time_each_step=0.66s, eta=0:0:13
2022-05-03 13:14:03 [INFO]	[TRAIN] Epoch 270 finished, loss=6.635105, lr=1e-06 .
2022-05-03 13:14:03 [INFO]	Start to evaluating(total_samples=121, total_steps=16)...


100%|██████████| 16/16 [00:08<00:00,  1.83it/s]


2022-05-03 13:14:12 [INFO]	[EVAL] Finished, Epoch=270, bbox_map=90.391628 .
2022-05-03 13:14:13 [INFO]	Model saved in output_yolov3_darknet53/epoch_270.
2022-05-03 13:14:13 [INFO]	Current evaluated best model in eval_dataset is epoch_240, bbox_map=94.7143975650474
